# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../output_data/weather.csv", dtype = "object", 
                         encoding="utf-8")
weather_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...,...
543,543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.6,8.05
544,544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.2,16.11
545,545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.0,8.05
546,546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Storing "Lat" and "Lng" as location variable
location = weather_df[["Lat", "Lng"]].astype(float)
#Storing "Humidity" as weight
weight = weather_df["Humidity"]

#Adding Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weight,
                                dissipating=False, max_intensity= 100,
                                point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_df.loc[(weather_df["Max Temp"].astype(float) < 80) & (weather_df["Max Temp"].astype(float) > 70) \
                                    & (weather_df["Wind Speed"].astype(float) < 10) \
                                    & (weather_df["Cloudiness"].astype(float) == 0)].dropna()
ideal_weather_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32
88,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.5
138,138,138,morondava,0,MG,1558378780,69,-20.3,44.28,79.08,9.06
139,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.9,1.59
176,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93
297,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32,
88,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.5,
138,138,138,morondava,0,MG,1558378780,69,-20.3,44.28,79.08,9.06,
139,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.9,1.59,
176,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93,
297,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17,


In [11]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # TODO: change this into hotel_df once hotel_df has other data
    lat = hotel_df["Lat"]
    lng = hotel_df["Lng"]
    
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_address = requests.get(base_url, params=params)
    hotel_address = hotel_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except:
        print("Missing Parameter Result... Skipping.")
hotel_df

Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.
Missing Parameter Result... Skipping.


,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.0,73.68,5.32,
88,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.5,
138,138,138,morondava,0,MG,1558378780,69,-20.3,44.28,79.08,9.06,
139,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.9,1.59,
176,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.2,6.93,
297,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'